{
  "cells": [
    // notebook cells data
  ],
  "metadata": {},
  "nbformat": 4,
  "nbformat_minor": 4
}


In [2]:
import pandas as pd
data = pd.read_csv(r'/Users/chenyarou/Desktop/IMT 574/IMT574/analysis_scope_final.csv', low_memory=False) 
data.head()

,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,...,EMPLOYER_STATE,EMPLOYER_CITY,NAICS_CODE,LAWFIRM_NAME_BUSINESS_NAME,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PW_WAGE_LEVEL,H_1B_DEPENDENT,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,Certified,2020-02-19,2020-02-26,H-1B,15-1199.00,"Computer Occupations, All Other",Y,1,1,0,...,CA,LA PALMA,541611,LAW OFFICE OF VACHHANI & ASSOCIATES,32.5,Hour,II,N,32.44,Hour
1,Certified,2020-03-12,2020-03-19,H-1B,17-2051.00,Civil Engineers,Y,1,1,0,...,CA,Pasadena,541330,Dornbaum & Peregoy,71500.0,Year,I,N,68162.00,Year
2,Certified,2020-04-19,2020-04-24,H-1B,15-1199.01,Software Quality Assurance Engineers and Testers,Y,1,1,0,...,MA,Southborough,541511,NaN,79934.0,Year,II,Y,79934.00,Year
3,Certified,2020-03-10,2020-03-17,H-1B,13-1111.00,Management Analysts,Y,1,1,0,...,MA,BOSTON,541611,Seyfarth Shaw LLP,165000.0,Year,III,N,112403.00,Year
4,Certified,2020-03-24,2020-03-31,H-1B,15-1131.00,Computer Programmers,Y,1,1,0,...,NC,Durham,54151,"Fakhoury Global Immigration, USA PC",70200.0,Year,II,N,70200.00,Year


### Data processing

1. process null values （公司和律所是否一对一）
2. duplicated rows
3. caculate wage unit from hour to year
4. unconsistent data type

# Data Cleaning

## 1. Drop null values

In [3]:
# drop the rows where the value of 'EMPLOYER_NAME', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY' is null
# since there are not many null values
data = data.dropna(subset=['EMPLOYER_NAME', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY'])
# drop the rows where the value of 'H_1B_DEPENDENT' is null
data = data.dropna(subset=['H_1B_DEPENDENT'])
# replace the null value with 'Unknown'
data['LAWFIRM_NAME_BUSINESS_NAME'] = data['LAWFIRM_NAME_BUSINESS_NAME'].fillna('Unknown')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766129 entries, 0 to 791858
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CASE_STATUS                 766129 non-null  object 
 1   RECEIVED_DATE               766129 non-null  object 
 2   DECISION_DATE               766129 non-null  object 
 3   VISA_CLASS                  766129 non-null  object 
 4   SOC_CODE                    766129 non-null  object 
 5   SOC_TITLE                   766129 non-null  object 
 6   FULL_TIME_POSITION          766129 non-null  object 
 7   TOTAL_WORKER_POSITIONS      766129 non-null  int64  
 8   NEW_EMPLOYMENT              766129 non-null  int64  
 9   CONTINUED_EMPLOYMENT        766129 non-null  int64  
 10  CHANGE_PREVIOUS_EMPLOYMENT  766129 non-null  int64  
 11  NEW_CONCURRENT_EMPLOYMENT   766129 non-null  int64  
 12  CHANGE_EMPLOYER             766129 non-null  int64  
 13  AMENDED_PETITION   

## 2 Calculate the PW_WAGE_LEVEL based on the United States PW WAGE LEVEL RULES

In [4]:
import numpy as np

# define the Prevailing Wage Level calculation function
def assign_wage_level(row):
    if pd.notnull(row["PW_WAGE_LEVEL"]):  # only process null values
        return row["PW_WAGE_LEVEL"]
    
    prevailing_wage = row["PREVAILING_WAGE"]
    actual_wage = row["WAGE_RATE_OF_PAY_FROM"]
    
    if pd.isnull(prevailing_wage) or pd.isnull(actual_wage):  
        return np.nan

    # Calculate the ratio of salary and prevailing wage
    ratio = actual_wage / prevailing_wage
    
    # Decide the  Prevailing Wage Level based on the OES percentile
    if ratio < 1.17:
        return "I"  # Level 1 (17th percentile)
    elif ratio < 1.34:
        return "II"  # Level 2 (34th percentile)
    elif ratio < 1.5:
        return "III"  # Level 3 (50th percentile)
    else:
        return "IV"  # Level 4 (above 50th percentile)

# fill the null values of PW_WAGE_LEVEL
data["PW_WAGE_LEVEL"] = data.apply(assign_wage_level, axis=1)

# check the result
data["PW_WAGE_LEVEL"].isnull().sum()

np.int64(0)

## 3 Convert the type of RECEIVED_DATE, DECISION_DATE to datetime and calculate the processing days

In [10]:
data["RECEIVED_DATE"] = pd.to_datetime(data["RECEIVED_DATE"], format='%Y-%m-%d', errors='coerce')
data["DECISION_DATE"] = pd.to_datetime(data["DECISION_DATE"], format='%Y-%m-%d', errors='coerce')
data["PROCESSING_DAYS"] = (data["DECISION_DATE"] - data["RECEIVED_DATE"]).dt.days

## 4 Specify and standardize the columns that need to be unified in uppercase

In [11]:
cols_to_upper = ["SOC_TITLE", "EMPLOYER_NAME", "EMPLOYER_CITY", "LAWFIRM_NAME_BUSINESS_NAME"]
data[cols_to_upper] = data[cols_to_upper].apply(lambda x: x.str.upper())

In [12]:
import re

def clean_string(s):
    return re.sub(r'[^\w\s]', '', s)

data['UNIFIED_EMPLOYER_NAME'] = data['EMPLOYER_NAME'].apply(clean_string)
data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'] = data['LAWFIRM_NAME_BUSINESS_NAME'].apply(clean_string)

data['UNIFIED_EMPLOYER_NAME'] = data['UNIFIED_EMPLOYER_NAME'].str.replace(' ', '')
data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'] = data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'].str.replace(' ', '')

data.head()

,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,...,LAWFIRM_NAME_BUSINESS_NAME,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PW_WAGE_LEVEL,H_1B_DEPENDENT,PREVAILING_WAGE,PW_UNIT_OF_PAY,PROCESSING_DAYS,UNIFIED_EMPLOYER_NAME,UNIFIED_LAWFIRM_NAME_BUSINESS_NAME
0,Certified,2020-02-19,2020-02-26,H-1B,15-1199.00,"COMPUTER OCCUPATIONS, ALL OTHER",Y,1,1,0,...,LAW OFFICE OF VACHHANI & ASSOCIATES,32.5,Hour,II,N,32.44,Hour,7,FRIENDLYFRANCHISEESCORPORATION,LAWOFFICEOFVACHHANIASSOCIATES
1,Certified,2020-03-12,2020-03-19,H-1B,17-2051.00,CIVIL ENGINEERS,Y,1,1,0,...,DORNBAUM & PEREGOY,71500.0,Year,I,N,68162.00,Year,7,TETRATECHINC,DORNBAUMPEREGOY
2,Certified,2020-04-19,2020-04-24,H-1B,15-1199.01,SOFTWARE QUALITY ASSURANCE ENGINEERS AND TESTERS,Y,1,1,0,...,UNKNOWN,79934.0,Year,II,Y,79934.00,Year,5,VIRTUSACORPORATION,UNKNOWN
3,Certified,2020-03-10,2020-03-17,H-1B,13-1111.00,MANAGEMENT ANALYSTS,Y,1,1,0,...,SEYFARTH SHAW LLP,165000.0,Year,III,N,112403.00,Year,7,THEBOSTONCONSULTINGGROUPINC,SEYFARTHSHAWLLP
4,Certified,2020-03-24,2020-03-31,H-1B,15-1131.00,COMPUTER PROGRAMMERS,Y,1,1,0,...,"FAKHOURY GLOBAL IMMIGRATION, USA PC",70200.0,Year,II,N,70200.00,Year,7,IBMCORPORATION,FAKHOURYGLOBALIMMIGRATIONUSAPC


## 5 Standardize the value of H_1B_DEPENDENT

In [13]:
data['H_1B_DEPENDENT'] = data['H_1B_DEPENDENT'].replace({'Yes': 'Y', 'No': 'N'})

## 6 Standardize the unit of wages

In [14]:
# define the convertion function
def convert_to_yearly_wage(row):
    if row['WAGE_UNIT_OF_PAY'] == 'Hour':
        return row['WAGE_RATE_OF_PAY_FROM'] * 2080  
    elif row['WAGE_UNIT_OF_PAY'] == 'Month':
        return row['WAGE_RATE_OF_PAY_FROM'] * 12  
    elif row['WAGE_UNIT_OF_PAY'] == 'Bi-Weekly':
        return row['WAGE_RATE_OF_PAY_FROM'] * 26  
    elif row['WAGE_UNIT_OF_PAY'] == 'Week':
        return row['WAGE_RATE_OF_PAY_FROM'] * 52  
    else:
        return row['WAGE_RATE_OF_PAY_FROM']  # 如果已经是年薪，直接返回

def convert_to_yearly_pw(row):
    if row['PW_UNIT_OF_PAY'] == 'Hour':
        return row['PREVAILING_WAGE'] * 2080 
    elif row['PW_UNIT_OF_PAY'] == 'Month':
        return row['PREVAILING_WAGE'] * 12  
    elif row['PW_UNIT_OF_PAY'] == 'Bi-Weekly':
        return row['PREVAILING_WAGE'] * 26  
    elif row['PW_UNIT_OF_PAY'] == 'Week':
        return row['PREVAILING_WAGE'] * 52 
    else:
        return row['PREVAILING_WAGE'] 

# apply the function
data['WAGE_RATE_OF_PAY_FROM'] = data.apply(convert_to_yearly_wage, axis=1)
data['PREVAILING_WAGE'] = data.apply(convert_to_yearly_pw, axis=1)

# update all the unit to 'Year'
data['WAGE_UNIT_OF_PAY'] = 'Year'  
data['PW_UNIT_OF_PAY'] = 'Year'  


data[['WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY']].head()


,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,67600.0,Year,67475.2,Year
1,71500.0,Year,68162.0,Year
2,79934.0,Year,79934.0,Year
3,165000.0,Year,112403.0,Year
4,70200.0,Year,70200.0,Year


## 7 Create the mapping of SOC_CODE and SOC_TITLE

In [15]:
data.to_csv('processed_data.csv', index=False)